In [20]:

import pandas as pd


validacion = pd.read_csv('logs_validacion.csv')

notas = pd.read_csv('notas.csv')

column_names_df = notas.columns


column_names_validacion = validacion.columns

# Print the column names
print("Column names of 'df':", column_names_df)
print("Column names of 'validacion':", column_names_validacion)

# Rename a single column
notas.rename(columns={'Username': 'username'}, inplace=True)

notas = notas.replace('Not Attempted', 0)

print(notas)


Column names of 'df': Index(['Unnamed: 0', 'Username', 'Grade', 'Grade Scaled',
       'Quiz 1: Uso de modelos matemáticos en la economía',
       'Quiz 2: Motivación y Función de Producción',
       'Quiz 3: Preferencias y Toma de Decisiones',
       'Quiz 4: Salario y Efectos',
       'Quiz 5: Explicando Diferencias y Conclusión',
       'Quiz 6: Introducción y conceptos iniciales',
       'Quiz 7: Prediciendo el resultado de un juego',
       'Quiz 8: Evaluación de Resultados',
       'Quiz 9: Determinación de Asignaciones', 'Quiz 10: Desigualdad',
       'Quiz 11: Conceptos centrales', 'Quiz 12: Profundizando lo aprendido',
       'Quiz 13: Conceptos centrales', 'Quiz 14: Profundizando lo aprendido',
       'Quiz 15: Conceptos centrales', 'Quiz 16: Profundizando lo aprendido',
       'Quiz 17: Introducción',
       'Quiz 18: Oferta, demanda y equilibrio de mercado',
       'Quiz 19: Cambios en la oferta y la demanda',
       'Quiz 20: Excedentes y la mano invisible',
       'Quiz 2

In [21]:
#ACÁ SE TOMA INFO DE LOG_TEST PARA SUPLIR LA DATA DE NOTAS


# Convert the 'time_column' to a datetime object
validacion['time'] = pd.to_datetime(validacion['time'])

# Create new columns for month, day, and time
validacion['month'] = validacion['time'].dt.month
validacion['day'] = validacion['time'].dt.day
validacion['hora'] = validacion['time'].dt.strftime('%H:%M:%S')




#----------------------------------

# DURACION DE SESIONES (una sesion es un día con registro de ingreso)

# Group the DataFrame by 'usuario', 'month', and 'day' and calculate the min and max time
grouped = validacion.groupby(['username', 'month', 'day'])['time'].agg(['min', 'max']).reset_index()

#
grouped['time_difference'] = (grouped['max'] - grouped['min']).dt.total_seconds() / 3600

# Merge the 'time_difference' column from the 'grouped' DataFrame into the 'eventos' DataFrame
validacion = validacion.merge(grouped[['username', 'month', 'day', 'time_difference']], on=['username', 'month', 'day'], how='left')



#----------------------------------

#ESTADISTICOS DE DURACION DE SESIONES

# Group the DataFrame by 'username' and calculate the average and standard deviation of 'time_difference'
grouped = validacion.groupby('username')['time_difference'].agg(['mean', 'std']).reset_index()

# Rename the columns for clarity
grouped.columns = ['username', 'average_time_difference', 'std_time_difference']

#Agregarlo a la data final
notas = notas.merge(grouped[['username', 'average_time_difference', 'std_time_difference']].drop_duplicates(), on='username', how='left')



#----------------------------------

#DURACION EN DÍAS DE SU USO DE EOL

# Extract the date part
validacion['date'] = validacion['time'].dt.date

# Group the DataFrame by 'usuario', 'month', and 'day' and calculate the min and max time
grouped = validacion.groupby(['username'])['date'].agg(['min', 'max']).reset_index()

grouped['min'] = pd.to_datetime(grouped['min'])
grouped['max'] = pd.to_datetime(grouped['max'])

# Calculate the time difference in hours
grouped['duracionEOL'] = (grouped['max'] - grouped['min']).dt.days

#Agregarlo a la data final
notas = notas.merge(grouped[['username', 'duracionEOL']].drop_duplicates(), on='username', how='left')




In [22]:

#----------------------------------

#FRECUENCIA DE SESIONES POR MES


distinct_months = validacion['month'].unique()

# Create a new dataframe to store the results
result_data = {'username': validacion['username']}
for month in distinct_months:
    result_data[month] = validacion['username'].map(
        validacion[validacion['month'] == month].groupby('username')['day'].nunique()
    )

# Convert the dictionary to a DataFrame
result_df = pd.DataFrame(result_data)

# Rename multiple columns
result_df.rename(columns={'8': 'Num Sesiones de Agosto', '9': 'Num Sesiones de Septiembre', '10': 'Num Sesiones de Noviembre'}, inplace=True)


result_df.fillna(0, inplace=True)

# Merge the 'result_df' with the 'df_notas' dataframe based on the 'username' column
notas = notas.merge(result_df, on='username', how='left')

# Print the updated 'eventos' DataFrame
print(notas)


        Unnamed: 0                                           username  Grade  \
0                3  05743498d061bf472df897f1e5bdda6d62e99312c952e3...   0.94   
1                3  05743498d061bf472df897f1e5bdda6d62e99312c952e3...   0.94   
2                3  05743498d061bf472df897f1e5bdda6d62e99312c952e3...   0.94   
3                3  05743498d061bf472df897f1e5bdda6d62e99312c952e3...   0.94   
4                3  05743498d061bf472df897f1e5bdda6d62e99312c952e3...   0.94   
...            ...                                                ...    ...   
196820         608  1e1188e0478a823fc3f411ef6baed0f6abc324d5b70e60...   0.51   
196821         608  1e1188e0478a823fc3f411ef6baed0f6abc324d5b70e60...   0.51   
196822         608  1e1188e0478a823fc3f411ef6baed0f6abc324d5b70e60...   0.51   
196823         610  d4e09736a16c403937d6e829e6cdae6ed0537277c2c34d...   0.00   
196824         613  714d13c3517b88defa5763d78fc0c840a4bc1ad6092c02...   0.00   

        Grade Scaled Quiz 1: Uso de mod

In [23]:
# Get unique values from a specific column
unique_values = validacion['grouped_event_type'].unique()
print(unique_values)

['/courses/(course-code)/course/'
 'edx.bi.course.upgrade.sidebarupsell.displayed'
 '/courses/(course-code)/course_wiki' 'edx.ui.lms.link_clicked'
 '/courses/(course-code)/wiki/(course-code)/'
 '/courses/(course-code)/progress'
 '/courses/(course-code)/jump_to/(block-code)@vertical@(uuid)'
 '/courses/(course-code)/courseware/(uuid1)/(uuid2)/(tab-n)'
 '/courses/(course-code)/xblock/block-v1:(block-code)@html+block@(uuid)/handler/publish_completion'
 '/courses/(course-code)/courseware/(uuid-code1)/(uuid-code2)/(some-path)'
 'edx.ui.lms.sequence.next_selected' 'load_video' 'play_video'
 'pause_video'
 '/courses/(course-code)/xblock/block-v1:(block-code)@sequential+block@(uuid)/handler/xmodule_handler/goto_position'
 '/courses/(course-code)/xblock/block-v1:(block-code)@sequential+block@(uuid)/handler/xmodule_handler/get_completion'
 'seq_goto' '/courses/(course-code)/discussion/(discussion-id)/'
 '/courses/(course-code)/(uuid-course)/' 'seek_video' 'seq_next'
 'edx.ui.lms.sequence.previous

In [26]:
#SE ENCUENTRAN LOS ESTADISTICOS DE LAS ACTIVIDADES QUE REALIZA EN CADA SESION
# IE CANTIDAD DE VECES QUE REALIZA ALGUNA ACTIVIDAD EN UNA SESION (MONTH, DAY)
# List of event types to create columns for
validacion_types = ['page_close', 'problem_graded', 'problem_check', 'problem_show', 'seg_prev',
               'seg_next', 'seg_goto', 'load_video', 'play_video', 'pause_video', 'speed_change_video']

# Iterate through the event types and create columns with boolean values
for validacion_type in validacion_types:
    validacion[validacion_type] = (validacion['grouped_event_type'] == validacion_type).astype(int)

grouped_eventos = validacion.groupby(['username', 'month', 'day'])[validacion_types].sum().reset_index()



# Group the sub_df DataFrame by 'username' and calculate the mean and standard deviation for each event type

agg_df = grouped_eventos.groupby('username').agg({
    'page_close': ['mean', 'std'],
    'problem_graded': ['mean', 'std'],
    'problem_check': ['mean', 'std'],
    'problem_show': ['mean', 'std'],
    'seg_prev': ['mean', 'std'],
    'seg_next': ['mean', 'std'],
    'seg_goto': ['mean', 'std'],
    'load_video': ['mean', 'std'],
    'play_video': ['mean', 'std'],
    'pause_video': ['mean', 'std'],
    'speed_change_video': ['mean', 'std']
}).reset_index()


# Aplanar las columnas de múltiples niveles en 'agg_df'
agg_df.columns = ['_'.join(col).rstrip() for col in agg_df.columns.values]


# Renombrar la columna en 'agg_df'
agg_df = agg_df.rename(columns={'username_': 'username'})

# Combinar el 'agg_df' modificado con el DataFrame 'notas'
notas = notas.merge(agg_df, on='username', how='left')

# Display the resulting 'notas' DataFrame


True False


In [27]:
#COUNT ACTIVIDADES PER CHAPTER (encontrar chapters que haya estudioado mas/menos)

grouped = validacion.groupby(['username', 'chapter'])['grouped_event_type'].count().reset_index()

pivoted_df = grouped.pivot(index='username', columns='chapter', values='grouped_event_type').fillna(0)

# Reset the index to make 'column1' a regular column
pivoted_df.reset_index(inplace=True)

# Rename columns for clarity
pivoted_df.columns.name = None

# Assuming 'notas' is the DataFrame where you want to add the columns
notas = notas.merge(pivoted_df, on='username', how='left')


In [31]:
#AGREGAR LABELS

# Verificar si la columna 'label' existe en 'validacion'
label_in_validacion = 'label' in validacion.columns

print(label_in_validacion)


#label_df = validacion[['username', 'label']].drop_duplicates(subset='username')


#merged_df = pd.merge(notas, label_df, on='username', how='inner')

#merged_df.name = notas

False


KeyError: "['label'] not in index"

In [32]:
#CREAMOS UN DICCIONARIO CON PARA PODER GUARDAR A LOS USUARIOS CON NOMBRES MAS PEQUEÑOS
import pandas as pd


# Genera nombres de usuario acortados automáticamente
notas['user_id'] = ['u' + str(i+1) for i in range(len(notas))]

# Crea un diccionario para mapear nombres acortados a nombres largos
diccionario_mapeo = dict(zip(notas['user_id'], notas['username']))

notas = notas.drop('username', axis=1)

column_order = ['user_id'] + [col for col in notas if col != 'user_id']
notas = notas[column_order]


#PARA ENCONTRAR A UN USUARIO SE USA LO SIGUIENTE
#nombre_acortado = 'u2'
#nombre_largo = diccionario_mapeo[nombre_acortado]

#print(f"Nombre largo correspondiente a '{nombre_acortado}': {nombre_largo}")


In [33]:
notas.sample(50)

,user_id,Unnamed: 0,Grade,Grade Scaled,Quiz 1: Uso de modelos matemáticos en la economía,Quiz 2: Motivación y Función de Producción,Quiz 3: Preferencias y Toma de Decisiones,Quiz 4: Salario y Efectos,Quiz 5: Explicando Diferencias y Conclusión,Quiz 6: Introducción y conceptos iniciales,...,speed_change_video_std,0,1,2,3,4,5,6,7,none_page
158465,u158466,436,0.98,6.8,1.0,1.0,1.0,1.0,1.0,1.0,...,2.578759,21.0,200.0,158.0,233.0,335.0,114.0,72.0,63.0,457.0
86236,u86237,280,0.96,6.7,1.0,1.0,1.0,0.0,1.0,1.0,...,1.976529,18.0,481.0,462.0,949.0,747.0,670.0,410.0,430.0,502.0
129438,u129439,386,1.00,7.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.142369,0.0,94.0,105.0,214.0,142.0,74.0,153.0,76.0,203.0
136614,u136615,412,0.99,6.9,0.75,1.0,1.0,1.0,1.0,1.0,...,0.744946,0.0,48.0,89.0,271.0,140.0,73.0,149.0,79.0,316.0
169452,u169453,475,0.73,5.0,0,1.0,1.0,1.0,1.0,1.0,...,1.069045,26.0,76.0,41.0,107.0,130.0,42.0,90.0,72.0,305.0
8250,u8251,31,1.00,7.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.755929,0.0,26.0,109.0,77.0,49.0,24.0,54.0,34.0,422.0
132917,u132918,395,0.86,6.0,0.75,1.0,1.0,1.0,1.0,1.0,...,1.974192,0.0,17.0,39.0,73.0,55.0,35.0,45.0,55.0,301.0
33145,u33146,109,0.96,6.7,0.75,1.0,1.0,1.0,1.0,1.0,...,0.000000,0.0,158.0,110.0,110.0,175.0,131.0,59.0,17.0,338.0
148243,u148244,424,0.99,6.9,1.0,1.0,1.0,1.0,1.0,1.0,...,6.939921,26.0,1557.0,2025.0,3185.0,118.0,408.0,329.0,57.0,553.0
92519,u92520,288,0.88,6.1,1.0,0.0,1.0,1.0,1.0,1.0,...,1.196486,26.0,410.0,249.0,769.0,556.0,523.0,520.0,380.0,493.0


In [34]:
notas.to_csv('data_para_validar.csv')

#PENDIENTE

- encontrar forma de incluir las variables de event type (conocer los distintos tipos de event type y sacar la frecuencia de cada uno en las sesiones y a partir de esto sacar estadisticos, ej en promedio cuantas veces pone pausa, a que vwlocidad ve ek video, repite mucho los mismos videos etc)
-quizas los estadísticos de la duracion de las sesiones podría ser especifico a cada mes
